In [ ]:
import pandas as pd
from prophet import Prophet
from google.colab import files
import xlrd
import io

uploaded = files.upload()


In [ ]:
df = pd.read_excel('/content/Recharge Data.xlsx')
df['ds'] = pd.to_datetime(df['ds'])

m = Prophet()
m.add_seasonality(name='weekly', period=7, fourier_order=5)
m.add_seasonality(name='monthly', period=30, fourier_order=8)
m.add_seasonality(name='quarterly', period=90, fourier_order=8)
m.add_country_holidays(country_name='PAK')

m.fit(df)

df_future = m.make_future_dataframe(periods=365)
forecast = m.predict(df_future)


In [ ]:
from prophet.diagnostics import cross_validation, performance_metrics

df_cv = cross_validation(m, initial = '365 days', period = '30 days', horizon = '30 days')

df_p = performance_metrics(df_cv)
df_p.to_excel('df_p.xlsx')
df_cv.to_excel('df_cv.xlsx')

In [ ]:
from google.colab import drive

forecast.to_excel('/content/drive/MyDrive/forecast.xlsx', index=False)

In [ ]:
!git clone https://github.com/Sannan27/glowing-adventure.git

%cd glowing-adventure

!git config user.name "abc"
!git config user.email "abc"

!git add .
!git commit -m"Recharge Forecast v 1.0"
!git push

In [9]:
%cd /content
!rm -rf glowing-adventure

/content
